# EXP-011: Optuna Per-Target Params + Стекинг

**Результат: LB 0.8472** (предыдущий лучший EXP-009: 0.8445, дельта +0.0027)

## Пайплайн
1. **Optuna** — подбор гиперпараметров XGBoost для каждого из 41 таргетов (100k, 3-fold, 20 trials)
2. **L1 OOF** — XGBoost 5-fold OOF с Optuna params + per-target feature selection (750k)
3. **Full Train + Test Inference** — XGBoost full train 750k → predict test 250k
4. **L2 Стекинг** — мета-модель XGBoost depth=2 на 41 OOF-фиче → финальный сабмит

## Результаты
- L1 OOF Macro AUC: **0.8407** (было 0.8352, +0.0055)
- L2 Meta OOF Macro AUC: **0.8423**
- **Public LB: 0.8472**

## Время
- Optuna: ~14 часов (A100, 100k × 41 таргетов × 20 trials)
- OOF: 135 мин (A100)
- Full train + test: 22 мин (A100)
- L2 стекинг: <1 мин

## Артефакты
- `optuna_best_params.json` — Optuna параметры для 41 таргета
- `xgb_oof_optuna.npy` — L1 OOF предсказания (750k, 41)
- `xgb_best_features_optuna.json` — отобранные фичи для каждого таргета
- `xgb_test_optuna.npy` — L1 test предсказания (250k, 41)
- `submission_optuna_stacking.parquet` — финальный сабмит

---
## Шаг 0: Настройка окружения

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna
import json, os, time, gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

from google.colab import drive
drive.mount('/content/drive')

DATA = '/content/drive/MyDrive/data_fusion'
print(f"XGBoost version: {xgb.__version__}")
print(f"CUDA available: {'cuda' in xgb.build_info().get('USE_CUDA', '')}" if hasattr(xgb, 'build_info') else 'check manually')

---
## Шаг 1: Optuna — подбор гиперпараметров (100k, 3-fold, 20 trials)

Каждый из 41 таргетов получает свои оптимальные параметры.
Пространство поиска: max_depth [4-10], lr [0.01-0.1], colsample [0.3-0.9], mcw [1-20], reg_alpha/lambda [1e-3..10], n_rounds [300-2000].

**Время: ~14 часов на A100.**
Чекпоинт сохраняется после каждого таргета — можно перезапускать Colab.

In [ ]:
# === Настройки Optuna ===
SAMPLE = 100000       # Подвыборка для быстрого поиска
N_FOLDS_OPTUNA = 3    # 3-fold для скорости
N_TRIALS = 20         # 20 trials на таргет
CHECKPOINT = f'{DATA}/optuna_best_params.json'

# === Загрузка данных ===
print("Загрузка данных...")
train_main = pd.read_parquet(f'{DATA}/train_main_features.parquet').head(SAMPLE)
train_extra = pd.read_parquet(f'{DATA}/train_extra_features.parquet').head(SAMPLE)
train_target = pd.read_parquet(f'{DATA}/train_target.parquet').head(SAMPLE)

X = train_main.merge(train_extra, on='customer_id', how='left').drop(columns=['customer_id'])
y = train_target.drop(columns=['customer_id'])
target_columns = y.columns.tolist()

X_np = np.ascontiguousarray(X.astype(np.float32).values)
y_np = y.values
del train_main, train_extra, train_target, X, y
gc.collect()
print(f"Данные: {X_np.shape}, таргетов: {len(target_columns)}")

# === Загрузка чекпоинта (если Colab перезапускался) ===
if os.path.exists(CHECKPOINT):
    with open(CHECKPOINT, 'r') as f:
        best_params_dict = json.load(f)
    print(f"Чекпоинт найден: {len(best_params_dict)} таргетов уже готово")
else:
    best_params_dict = {}

# === Базовые параметры XGBoost ===
base_params = {
    'objective': 'binary:logistic', 'eval_metric': 'auc',
    'tree_method': 'hist', 'device': 'cuda', 'seed': 42, 'verbosity': 0,
}

In [ ]:
def eval_oof(params, n_rounds, X_data, y_target):
    """3-fold OOF AUC для Optuna."""
    skf = StratifiedKFold(n_splits=N_FOLDS_OPTUNA, shuffle=True, random_state=42)
    oof = np.zeros(len(y_target))
    for tr_idx, val_idx in skf.split(X_data, y_target):
        dtrain = xgb.DMatrix(X_data[tr_idx], label=y_target[tr_idx])
        dval = xgb.DMatrix(X_data[val_idx], label=y_target[val_idx])
        model = xgb.train(params, dtrain, num_boost_round=n_rounds, verbose_eval=False)
        oof[val_idx] = model.predict(dval)
    return roc_auc_score(y_target, oof)


# === Optuna для каждого таргета ===
print(f"\n{'='*70}")
print(f"{'Таргет':<15} {'Default':<10} {'Optuna':<10} {'Дельта':<10} {'Время':<8} {'Лучшие параметры'}")
print(f"{'='*70}")

start_total = time.time()

for t_idx, target_col in enumerate(target_columns):
    # Пропускаем уже обученные (чекпоинт)
    if target_col in best_params_dict:
        p = best_params_dict[target_col]
        print(f"[{t_idx+1:2d}/41] {target_col:<15} ПРОПУСК (чекпоинт, Optuna AUC: {p.get('best_auc', '?')})")
        continue

    y_target = y_np[:, t_idx]
    n_pos = int(y_target.sum())
    start_t = time.time()

    # Защита: слишком мало позитивов
    if n_pos < N_FOLDS_OPTUNA * 2:
        best_params_dict[target_col] = {
            'max_depth': 6, 'learning_rate': 0.05, 'n_rounds': 500,
            'subsample': 0.8, 'colsample_bytree': 0.8,
            'min_child_weight': 1, 'reg_alpha': 0.01, 'reg_lambda': 1.0,
            'best_auc': 0.5, 'note': 'too_few_positives'
        }
        print(f"[{t_idx+1:2d}/41] {target_col:<15} ПРОПУСК (pos={n_pos})")
        continue

    # Default AUC
    default_params = base_params.copy()
    default_params.update({
        'max_depth': 6, 'learning_rate': 0.05,
        'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 5
    })
    auc_default = eval_oof(default_params, 500, X_np, y_target)

    # Optuna objective
    def objective(trial):
        params = base_params.copy()
        params['max_depth'] = trial.suggest_int('max_depth', 4, 10)
        params['learning_rate'] = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)
        params['subsample'] = trial.suggest_float('subsample', 0.5, 0.9)
        params['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.3, 0.9)
        params['min_child_weight'] = trial.suggest_int('min_child_weight', 1, 20)
        params['reg_alpha'] = trial.suggest_float('reg_alpha', 1e-3, 10, log=True)
        params['reg_lambda'] = trial.suggest_float('reg_lambda', 1e-3, 10, log=True)
        n_rounds = trial.suggest_int('n_rounds', 300, 2000)
        return eval_oof(params, n_rounds, X_np, y_target)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=N_TRIALS)

    best_p = study.best_params
    auc_optuna = study.best_value

    # Сохраняем лучшие параметры
    best_params_dict[target_col] = {
        'max_depth': best_p['max_depth'],
        'learning_rate': best_p['learning_rate'],
        'n_rounds': best_p['n_rounds'],
        'subsample': best_p['subsample'],
        'colsample_bytree': best_p['colsample_bytree'],
        'min_child_weight': best_p['min_child_weight'],
        'reg_alpha': best_p['reg_alpha'],
        'reg_lambda': best_p['reg_lambda'],
        'best_auc': auc_optuna,
        'default_auc': auc_default,
    }

    # Чекпоинт после каждого таргета
    with open(CHECKPOINT, 'w') as f:
        json.dump(best_params_dict, f, indent=2)

    elapsed = (time.time() - start_t) / 60
    delta = auc_optuna - auc_default
    print(f"[{t_idx+1:2d}/41] {target_col:<15} {auc_default:.4f}    {auc_optuna:.4f}    {delta:+.4f}    {elapsed:.1f}м  "
          f"d={best_p['max_depth']} lr={best_p['learning_rate']:.3f} r={best_p['n_rounds']} "
          f"col={best_p['colsample_bytree']:.2f} mcw={best_p['min_child_weight']}")

elapsed_total = (time.time() - start_total) / 60
print(f"\n{'='*70}")
print(f"Готово! Время: {elapsed_total:.1f} мин")
print(f"Параметры сохранены: {CHECKPOINT}")

# Итоговая статистика
defaults = [v['default_auc'] for v in best_params_dict.values() if 'default_auc' in v]
optimized = [v['best_auc'] for v in best_params_dict.values()]
print(f"Средний Default AUC: {np.mean(defaults):.4f}")
print(f"Средний Optuna AUC:  {np.mean(optimized):.4f}")
print(f"Средний прирост:     {np.mean(optimized) - np.mean(defaults):+.4f}")

---
## Шаг 2: L1 OOF генерация (750k, 5-fold, Optuna params + Feature Selection)

Для каждого таргета:
1. Черновик: обучаем XGBoost на 100 iter → feature importance → отбираем топ-95% cumulative gain
2. Чистовик: 5-fold OOF с Optuna параметрами на отобранных фичах

**Время: ~135 мин на A100.**

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import json, time, gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from google.colab import drive
drive.mount('/content/drive')

DATA = '/content/drive/MyDrive/data_fusion'

# === Загрузка данных ===
print("Загрузка данных...")
train_main = pd.read_parquet(f'{DATA}/train_main_features.parquet')
train_extra = pd.read_parquet(f'{DATA}/train_extra_features.parquet')
train_target = pd.read_parquet(f'{DATA}/train_target.parquet')

X_train_df = train_main.merge(train_extra, on='customer_id', how='left').drop(columns=['customer_id'])
y_train_np = train_target.drop(columns=['customer_id']).values
target_columns = train_target.drop(columns=['customer_id']).columns.tolist()
feature_columns = X_train_df.columns.tolist()

X_train_np = np.ascontiguousarray(X_train_df.astype(np.float32).values)
del train_main, train_extra, train_target, X_train_df; gc.collect()

# === Загрузка Optuna параметров ===
with open(f'{DATA}/optuna_best_params.json', 'r') as f:
    optuna_params = json.load(f)
print(f"Optuna параметры: {len(optuna_params)} таргетов")

# === Настройки OOF ===
N_FOLDS = 5
N_ROUNDS_DRAFT = 100      # Итераций для черновика (feature selection)
GAIN_THRESHOLD = 0.95     # Порог cumulative gain
OOF_PATH = f'{DATA}/xgb_oof_optuna.npy'
FEAT_PATH = f'{DATA}/xgb_best_features_optuna.json'

oof_predictions = np.zeros_like(y_train_np, dtype=np.float32)
best_features_dict = {}

# Мастер-матрица для черновиков (QuantileDMatrix — быстрее для повторных обучений)
print("Построение мастер-матрицы...")
dtrain_full_draft = xgb.QuantileDMatrix(X_train_np)
print(f"Данные: {X_train_np.shape}, старт OOF...\n")

In [ ]:
start_total = time.time()

for t_idx, target_col in enumerate(target_columns):
    start_t = time.time()
    y_target = y_train_np[:, t_idx]
    n_pos = int(y_target.sum())

    # Загружаем Optuna параметры для этого таргета
    p = optuna_params.get(target_col, {})

    current_params = {
        'objective': 'binary:logistic', 'eval_metric': 'auc',
        'tree_method': 'hist', 'device': 'cuda', 'seed': 42, 'verbosity': 0,
        'max_depth': int(p.get('max_depth', 6)),
        'learning_rate': p.get('learning_rate', 0.05),
        'subsample': p.get('subsample', 0.8),
        'colsample_bytree': p.get('colsample_bytree', 0.8),
        'min_child_weight': int(p.get('min_child_weight', 5)),
        'reg_alpha': p.get('reg_alpha', 0.01),
        'reg_lambda': p.get('reg_lambda', 1.0),
    }
    n_rounds = int(p.get('n_rounds', 500))

    # Защита: слишком мало позитивов
    current_folds = min(N_FOLDS, n_pos) if n_pos > 0 else 0
    if current_folds < 2:
        oof_predictions[:, t_idx] = 0.0001
        best_features_dict[target_col] = feature_columns
        print(f"[{t_idx+1:2d}/41] {target_col:<15} ПРОПУСК (pos={n_pos})")
        continue

    # --- Черновик: feature selection (cumulative gain 95%) ---
    dtrain_full_draft.set_label(y_target)
    draft_params = current_params.copy()
    model_draft = xgb.train(draft_params, dtrain_full_draft,
                            num_boost_round=N_ROUNDS_DRAFT, verbose_eval=False)

    imp = model_draft.get_score(importance_type='gain')
    imp_sorted = sorted(imp.items(), key=lambda x: x[1], reverse=True)

    if not imp_sorted:
        selected_indices = list(range(X_train_np.shape[1]))
    else:
        selected_indices = []
        total_gain = sum(v for _, v in imp_sorted)
        cum_gain = 0
        for feat, gain in imp_sorted:
            cum_gain += gain
            selected_indices.append(int(feat[1:]))  # 'f123' -> 123
            if cum_gain / total_gain >= GAIN_THRESHOLD:
                break
        if len(selected_indices) < 10:
            selected_indices = [int(f[1:]) for f, _ in imp_sorted[:10]]

    best_features_dict[target_col] = [feature_columns[i] for i in selected_indices]

    # --- Чистовик: K-Fold OOF на отобранных фичах с Optuna параметрами ---
    X_target = X_train_np[:, selected_indices].copy(order='C')
    skf = StratifiedKFold(n_splits=current_folds, shuffle=True, random_state=42)

    for train_idx, val_idx in skf.split(X_target, y_target):
        dtrain = xgb.QuantileDMatrix(X_target[train_idx], label=y_target[train_idx])
        dval = xgb.QuantileDMatrix(X_target[val_idx], label=y_target[val_idx], ref=dtrain)
        model = xgb.train(current_params, dtrain,
                          num_boost_round=n_rounds, verbose_eval=False)
        oof_predictions[val_idx, t_idx] = model.predict(dval)

    try:
        auc = roc_auc_score(y_target, oof_predictions[:, t_idx])
    except:
        auc = 0.5

    elapsed = (time.time() - start_t) / 60
    print(f"[{t_idx+1:2d}/41] {target_col:<15} OOF AUC: {auc:.4f} | "
          f"Фичей: {len(selected_indices):<4} | {elapsed:.1f} мин")

    # Чекпоинт каждые 5 таргетов
    if (t_idx + 1) % 5 == 0:
        np.save(OOF_PATH, oof_predictions)
        with open(FEAT_PATH, 'w') as f:
            json.dump(best_features_dict, f)

# === Итоги ===
elapsed_total = (time.time() - start_total) / 60
oof_aucs = []
for i, col in enumerate(target_columns):
    try:
        oof_aucs.append(roc_auc_score(y_train_np[:, i], oof_predictions[:, i]))
    except:
        oof_aucs.append(0.5)

print(f"\nГотово! Время: {elapsed_total:.1f} мин")
print(f"OOF Macro AUC: {np.mean(oof_aucs):.4f} (было 0.8352 в EXP-009)")

np.save(OOF_PATH, oof_predictions)
with open(FEAT_PATH, 'w') as f:
    json.dump(best_features_dict, f)
print(f"Сохранено: {OOF_PATH}, {FEAT_PATH}")

---
## Шаг 3: Full Train + Test Inference (750k → 250k)

Обучаем XGBoost на полных 750k (без CV) с Optuna params + отобранными фичами.
n_rounds = Optuna n_rounds × 1.2 (больше данных → можно чуть больше итераций).

**Время: ~22 мин на A100.**

In [ ]:
import json
import numpy as np
import pandas as pd
import xgboost as xgb
import time

# === Загрузка данных ===
train_main = pd.read_parquet(f'{DATA}/train_main_features.parquet')
train_extra = pd.read_parquet(f'{DATA}/train_extra_features.parquet')
test_main = pd.read_parquet(f'{DATA}/test_main_features.parquet')
test_extra = pd.read_parquet(f'{DATA}/test_extra_features.parquet')
targets = pd.read_parquet(f'{DATA}/train_target.parquet')

train = train_main.merge(train_extra, on='customer_id')
test = test_main.merge(test_extra, on='customer_id')

target_cols = [c for c in targets.columns if c.startswith('target_')]
feature_cols = [c for c in train.columns if c != 'customer_id']

X_train = train[feature_cols]
X_test = test[feature_cols]

# Загружаем Optuna params и отобранные фичи
with open(f'{DATA}/optuna_best_params.json', 'r') as f:
    optuna_params = json.load(f)
with open(f'{DATA}/xgb_best_features_optuna.json', 'r') as f:
    best_features = json.load(f)

print(f"Таргетов: {len(target_cols)}, Optuna params: {len(optuna_params)}, Feature sets: {len(best_features)}")

# === Full train + predict test ===
test_preds = np.zeros((len(X_test), len(target_cols)))
start = time.time()

for i, col in enumerate(target_cols):
    t0 = time.time()
    y = targets[col].values

    # Фичи для этого таргета
    feats = best_features.get(col, feature_cols)

    # Optuna params
    if col in optuna_params:
        p = optuna_params[col]
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'tree_method': 'hist',
            'device': 'cuda',
            'max_depth': p.get('max_depth', 6),
            'learning_rate': p.get('learning_rate', 0.05),
            'colsample_bytree': p.get('colsample_bytree', 0.8),
            'min_child_weight': p.get('min_child_weight', 5),
            'reg_alpha': p.get('reg_alpha', 0),
            'reg_lambda': p.get('reg_lambda', 1),
            'subsample': 0.8,
            'verbosity': 0,
        }
        n_rounds = p.get('n_rounds', 500)
    else:
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'tree_method': 'hist',
            'device': 'cuda',
            'max_depth': 6,
            'learning_rate': 0.05,
            'min_child_weight': 5,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'verbosity': 0,
        }
        n_rounds = 500

    # Full train (750k) — без early stopping, +20% итераций
    n_rounds_full = int(n_rounds * 1.2)

    dtrain = xgb.DMatrix(X_train[feats], label=y)
    dtest = xgb.DMatrix(X_test[feats])

    model = xgb.train(params, dtrain, num_boost_round=n_rounds_full)
    test_preds[:, i] = model.predict(dtest)

    elapsed = time.time() - t0
    if elapsed > 60:
        print(f"[{i+1:2d}/41] {col:20s} n_rounds={n_rounds_full:4d} | Фичей: {len(feats):4d} | {elapsed/60:.1f} мин")
    else:
        print(f"[{i+1:2d}/41] {col:20s} n_rounds={n_rounds_full:4d} | Фичей: {len(feats):4d} | {elapsed:.0f} сек")

    # Checkpoint каждые 10 таргетов
    if (i + 1) % 10 == 0:
        np.save(f'{DATA}/xgb_test_optuna_checkpoint.npy', test_preds)

np.save(f'{DATA}/xgb_test_optuna.npy', test_preds)
total = (time.time() - start) / 60
print(f"\nГотово! Время: {total:.1f} мин")
print(f"Сохранено: xgb_test_optuna.npy ({test_preds.shape})")

---
## Шаг 4: L2 Стекинг + Сабмит

Мета-модель XGBoost (depth=2, 100 iter) на 41 OOF-фиче.
L2 учит корреляции между таргетами (группы продуктов, антагонист target_10_1).

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# === Загрузка ===
oof = np.load(f'{DATA}/xgb_oof_optuna.npy')        # (750k, 41)
test_preds = np.load(f'{DATA}/xgb_test_optuna.npy') # (250k, 41)
targets = pd.read_parquet(f'{DATA}/train_target.parquet')
test_main = pd.read_parquet(f'{DATA}/test_main_features.parquet')

target_cols = [c for c in targets.columns if c.startswith('target_')]
y_true = targets[target_cols].values

print(f"OOF: {oof.shape}, Test: {test_preds.shape}, Targets: {y_true.shape}")
print(f"L1 OOF Macro AUC: {roc_auc_score(y_true, oof, average='macro'):.6f}")

# === L2 мета-модель ===
meta_oof = np.zeros_like(oof)
meta_test = np.zeros_like(test_preds)

l2_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',
    'max_depth': 2,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'verbosity': 0,
}

for i, col in enumerate(target_cols):
    y = y_true[:, i]
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    test_fold_preds = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(oof, y)):
        dtrain = xgb.DMatrix(oof[tr_idx], label=y[tr_idx])
        dval = xgb.DMatrix(oof[val_idx], label=y[val_idx])
        dtest = xgb.DMatrix(test_preds)

        model = xgb.train(
            l2_params, dtrain,
            num_boost_round=100,
            evals=[(dval, 'val')],
            early_stopping_rounds=20,
            verbose_eval=False
        )

        meta_oof[val_idx, i] = model.predict(dval)
        test_fold_preds.append(model.predict(dtest))

    meta_test[:, i] = np.mean(test_fold_preds, axis=0)

meta_auc = roc_auc_score(y_true, meta_oof, average='macro')
print(f"\nL2 Meta OOF Macro AUC: {meta_auc:.6f}")

# === Per-target сравнение ===
print(f"\n{'Таргет':20s} {'L1 OOF':>8s} {'L2 Meta':>8s} {'Дельта':>8s}")
for i, col in enumerate(target_cols):
    l1 = roc_auc_score(y_true[:, i], oof[:, i])
    l2 = roc_auc_score(y_true[:, i], meta_oof[:, i])
    print(f"{col:20s} {l1:8.4f} {l2:8.4f} {l2-l1:+8.4f}")

# === Сабмит ===
submit = pd.DataFrame({'customer_id': test_main['customer_id'].values.astype(np.int32)})
for i, col in enumerate(target_cols):
    submit_col = col.replace('target_', 'predict_')
    submit[submit_col] = meta_test[:, i].astype(np.float64)

submit.to_parquet(f'{DATA}/submission_optuna_stacking.parquet', index=False)
print(f"\nСабмит: {submit.shape}")
print(f"Колонки: {submit.columns.tolist()[:3]}...")
print(f"dtypes: customer_id={submit['customer_id'].dtype}, preds={submit.iloc[:, 1].dtype}")
print(f"\n>>> Public LB: 0.8472 <<<")